# Vectara Guide: APIv2

In early June 2024, we [announced](https://vectara.com/blog/introducing-vectaras-v2-api/) a new API v2 for Vectara.

Vectara’s API has been rich and capable in [APIv1](https://docs.vectara.com/docs/1.0/), but it was somewhat complex to work with. We have heard this from many developers, and have worked hard over the last few months go improve that, resulting in the launch of [APIv2](https://docs.vectara.com/docs) - a new and improved API.

This new version provides APIs that are much more RESTful, and follow the [resource-oriented design](https://cloud.google.com/apis/design/resources) methodology. 

In this notebook we explore the new APIs, and demonstrate the differences between APIv1 and APIv2 for some of the key operations.

## Understanding Vectara API V2

Being more resource-oriented, API v2 follows the object-oriented URL structure. This design philosophy makes using APIs much easier by integrating a few key ideas:
1. **Entities as Resources**: Entities like documents, corpora, or api keys are defined as resources
2. **Nouns not Verbs**: Resource names are nouns (e.g. "corpus") and not verbs (e.g. "getKeys")
3. **Correct use of HTTP**: Using the standard HTTP verbs in a way that is consistent with their intent
- *GET*: Retrieve a resource or a collection of resources
- *POST*: Create a new resource
- *PUT*: Update an existing resource
- *DELETE*: Remove a resource
- *PATCH*: Partially update a resource
4. **Hierarchy**: Reflect the hierarchical nature of resources (e.g., /corpora/{corpusKey}/documents/)
5. **Error**: Last but not least, make good of HTTP error codes in a proper manner

Aside from this big shift into resource-oriented design, here are a few other key changes:
1. **Updated defaults**: We've take the opportunity to update some of the default values in the API where it makes sense
2. **URL encoding**: In APIv2 many requests include more information in the URL, and thus you might need to percent-encode those to ensure they are valid in the URL

## Getting Started

To run this notebook, you will need your own API credentials, which you can obtain by [setting up](https://console.vectara.com/signup?utm_source=vectara&utm_medium=signup&utm_term=DevRel&utm_content=example-notebooks&utm_campaign=vectara-signup-DevRel-example-notebooks) a Vectara account and create a corpus. You can follow our [Quick Start](https://docs.vectara.com/docs/quickstart) guide for help with this process. Once you do this, you can specify all of the required environment variables for this notebook using one of the following methods:
1. You can define all of the variables in the cell below (e.g. please replace `"YOUR_CUSTOMER_ID"` with your actual customer ID)
2. You can define all of the variables in your terminal and then run this notebook (e.g. if the corpus you want to work with has ID 3, you would run `export corpus_id='3'`). Don't forget to comment out the below cell before you run the notebook.

In [1]:
#import os
#os.environ['customer_id'] = "YOUR_CUSTOMER_ID"
#os.environ['personal_api_key'] = "YOUR_PERSONAL_API_KEY"
#os.environ['corpus_id'] = "YOUR_CORPUS_ID"
#os.environ['corpus_key'] = "YOUR_CORPUS_KEY"
#os.environ['corpus_api_key'] = "YOUR_CORPUS_API_KEY"   # You will want this key to have both QueryService and IndexService

In [2]:
# Set all variables to corresponding os-environment variables
import os

customer_id = os.getenv('customer_id')
personal_api_key = os.getenv('personal_api_key')
corpus_id = os.getenv('corpus_id')
corpus_key = os.getenv('corpus_key')
corpus_api_key = os.getenv('corpus_api_key')

Here's a brief explanation of all the required parameters:
1. Your customer ID is the unique ID number associated with your Vectara account.
2. Your personal API key is used for general operations on your account. It has permissions to everything that your role can access.
3. Your corpus ID is a unique identifier for your corpus, expressed as an integer (This is generated for you automatically when you create a new corpus).
4. Your corpus key is also a unique identifier for the same corpus, but you can choose to call this whatever you would like. You'll notice throughout this notebook that APIv1 uses the corpus ID to identify your corpus and APIv2 uses the corpus key, giving you more freedom to choose how to refer to your corpus.
5. Your corpus API key is the API key that can only perform operations related to this specific corpus. It can help you add documents to the corpus (if it has IndexService) and query documents (if it has QueryService). For this notebook, you will want an API key that has both IndexService and QueryService.

# Adding Data to Vectara

## File Upload

Let's start by uploading a document to our corpus, comparing the methods for APIv1 and APIv2. 

We are going to upload the Vectara's [Pet Policy](../data/pet_policy.pdf) to our corpus, which is available in this repository's data folder.

In [3]:
# Using APIv1

import requests

url = f"https://api.vectara.io/v1/upload?c={customer_id}&o={corpus_id}"

payload={}
files=[
  ('file',('pet_policy',open('../data/pet_policy.pdf','rb'),'application/octet-stream'))
]
headers = {
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload, files=files)
res = response.json()
print(res)

{'response': {'status': {}, 'quotaConsumed': {'numChars': '9215', 'numMetadataChars': '5803'}}}


In [4]:
# Delete the document so that we can properly see the output message when using APIv2

import requests

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/documents/pet_policy"

payload={}
headers = {
  'x-api-key': personal_api_key
}

response = requests.request("DELETE", url, headers=headers, data=payload)

In [5]:
# Using APIv2

import requests

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/upload_file"

payload={}
files=[
  ('file',('pet_policy',open('../data/pet_policy.pdf','rb'),'application/octet-stream'))
]
headers = {
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload, files=files)
res = response.json()
print(res)
print(response.status_code)

{'id': 'pet_policy', 'metadata': {'Producer': 'Skia/PDF m118 Google Docs Renderer', 'Title': 'Employee Handbook - Company Pet Policy'}, 'storage_usage': {'bytes_used': 9215, 'metadata_bytes_used': 5803}}
201


Note that in both APIv1 and APIv2, the Python code example from the [API playground](https://docs.vectara.com/docs/rest-api/upload-file) will include `/path/to/file` and you need to replace it with the actual path to your file, as shown for Python in the example above:
``` Python
files=[
  ('file',('pet_policy',open('../data/pet_policy.pdf','rb'),'application/octet-stream'))
]
```

## Indexing

Now let's see how to add a text document to a corpus in APIv1 and APIv2. 

Unlike File Upload, with indexing we explicitly define the sections and text of the document for indexing. Indexing also allows you to specify section metadata that can be used for filtering search information. 

We will add a document that contains some of William Shakespeare's best work, following the same information provided in the example found [here](https://docs.vectara.com/docs/api-reference/indexing-apis/file-upload/format-for-upload).

In [6]:
# Using APIv1

import requests
import json

url = "https://api.vectara.io/v1/index"

payload = json.dumps({
  "customerId": customer_id,
  "corpusId": corpus_id,
  "document": {
    "documentId": "selected-works-of-shakespeare",
    "title": "William Shakespeare, Greatest Hits",
    "metadataJson": "{\"timespan\": \"26 April 1564---23 April 1616\", \"stars\": 5, \"author\": \"William Shakespeare\"}",
    "section": [
      {
        "title": "King Lear",
        "text": "Synopsis: King Lear, intending to divide his power and kingdom among his three daughters, demands public professions of their love. His youngest daughter, ...",
        "section": [
          {
            "title": "Act I",
            "text": "KENT: I thought the king had more affected the Duke of Albany than Cornwall.\nGLOUCESTER: It did always seem so to us...",
            "metadataJson": "{\"stage-instructions\": \"Enter KENT, GLOUCESTER, and EDMUND\"}"
          },
          {
            "title": "Act II",
            "text": "EDMUND: Save thee, Curan. ...",
            "metadataJson": "{\"stage-instructions\": \"Enter EDMUND, and CURAN meets him\"}"
          }
        ]
      },
      {
        "title": "Antony and Cleopatra",
        "text": "PHILO: Nay, but this dotage of our general's\nO'erflows the measure: those his goodly eyes, ..."
      }
    ]
  }
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'customer-id': customer_id,
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(res)

{'status': {'code': 'OK', 'statusDetail': '', 'cause': None}, 'quotaConsumed': {'numChars': '468', 'numMetadataChars': '1005'}}


In [7]:
# Delete the document so that we can properly see the output message when using APIv2

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/documents/selected-works-of-shakespeare"

payload={}
headers = {
  'x-api-key': personal_api_key
}

response = requests.request("DELETE", url, headers=headers, data=payload)

In [8]:
# Using APIv2

import requests
import json

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/documents"

payload = json.dumps({
  "id": "selected-works-of-shakespeare",
  "type": "structured",
  "title": "William Shakespeare, Greatest Hits",
  "metadata": {
    "timespan": "26 April 1564---23 April 1616",
    "stars": 5,
    "author": "William Shakespeare"
  },
  "sections": [
    {
      "title": "King Lear",
      "text": "Synopsis: King Lear, intending to divide his power and kingdom among his three daughters, demands public professions of their love. His youngest daughter, ...",
      "sections": [
        {
          "title": "Act I",
          "text": "KENT: I thought the king had more affected the Duke of Albany than Cornwall.\nGLOUCESTER: It did always seem so to us...",
          "metadata": {
            "stage-instructions": "Enter KENT, GLOUCESTER, and EDMUND"
          }
        },
        {
          "title": "Act II",
          "text": "EDMUND: Save thee, Curan. ...",
          "metadata": {
            "stage-instructions": "Enter EDMUND, and CURAN meets him"
          }
        }
      ]
    },
    {
      "title": "Antony and Cleopatra",
      "text": "PHILO: Nay, but this dotage of our general's\nO'erflows the measure: those his goodly eyes, ..."
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(res)

{'id': 'selected-works-of-shakespeare', 'metadata': {'timespan': '26 April 1564---23 April 1616', 'stars': 5, 'author': 'William Shakespeare'}, 'storage_usage': {'bytes_used': 468, 'metadata_bytes_used': 1005}}


A few things to notice:
1. The overall structure of the payload has changed. In APIv2, there is no longer a `document` section. Also, you do not need to specify your `customer_id` or `corpus_id`. These are implemented through the URL and header fields.
2. While APIv1 uses `metadataJson` to store document and section metadata, which needs to be a string representation of the metadata dictionary. In contrast, APIv2 simplifies this and uses `metadata` which allows you to just use a dictionary of metadata for each document and section. The APIv2 layout is better organized for users and eliminates the need to escape quotations.
3. Other field names have changed as well, such as: `documentId` -> `id` and `section` -> `sections`.
4. APIv2 now enforces that each `section` has `text` (this argument was optional for APIv1). For both versions, all other `section` parameters are optional.
5. For the APIv2 example, our code uses the `structured` document type. This is the most commonly used type when indexing a document because it allows users to divide the document by the explicit sections in the document and let Vectara perform the chunking of text for the best model performance. We also support the `core` document type in which you as the developer explicitly define how the document is chunked. You can read more about the different types of documents [here](https://docs.vectara.com/docs/api-reference/indexing-apis/indexing#core-document-object-definition).

# Querying

## Simple Corpus Query

We start with search only, and we'll see how to add generation later on. 

If we want to execute a simple semantic search operation, APIv2 provides a simple way to run this query using URL parameters.

Let's look at an example using APIv1 vs APIv2:

In [9]:
# Using APIv1

import requests
import json

url = "https://api.vectara.io/v1/query"
query_str = "Are pets allowed at Vectara?"
payload = json.dumps({
  "query": [
    {
      "query": query_str,
      "start": 0,
      "numResults": 10,
      "contextConfig": {
        "sentencesBefore": 0,
        "sentencesAfter": 0
      },
      "corpusKey": [
        {
          "customerId": customer_id,
          "corpusId": corpus_id
        }
      ],
    }
  ]
})

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}
response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()

print("Here's the top 3 search results:")
for i in range(3):
    print(f"{i+1}. {res['responseSet'][0]['response'][i]['text']}")

Here's the top 3 search results:
1. At Vectara, we pride ourselves on thinking outside the box, and our pet policy is no exception.
2. We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.
3. ●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.


In [10]:
# Using APIv2: simple query

import requests
import urllib.parse

encoded_query_str = urllib.parse.quote(query_str)
url = f"https://api.vectara.io/v2/corpora/{corpus_key}/query?query={encoded_query_str}"
payload={}
headers = {
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("GET", url, headers=headers, data=payload)
res = response.json()

print("Here's the top 3 search results:")
for i in range(3):
    print(f"{i+1}. {res['search_results'][i]['text']}")

Here's the top 3 search results:
1. We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.
2. At Vectara, we pride ourselves on thinking outside the box, and our pet policy is no exception.
3. ●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.


A few things here are new:
1. With APIv1, we use a POST operator, using the same general command structure as any other type of query in APIv1 (see others below). It is also required to specify `numResults` and a list of `corpusKey` even though we only want to search a single corpus. With APIv2, we use a completely separate command. Instead of using a POST, which is typically used to create a new resource, we use the more appropriate GET operator to simply pull information from our corpus. This is accomplished in a concise manner by simply encoding the URL with the corpus key and the query. That's it! Note how we used the `quote` method of urllib.parse to ensure the query is encoded properly.
3. The APIv2 URL is as you expect from a resource-oriented design: identifying the corpus as `/corpora/{corpus_key}` and then running the query on that corpus. That is also why we don't need to specify the `corpusKey` as we had to do with APIv1.

## Corpus Query

The simple corpus query provides a simple way to run a search but it does support the full set of search parameters (like hybrid search, sentencesBefore, sentencesAfter, or reranking), not does it allow for summarization. Let's see how a full RAG query works for a single corpus, again comparing APIv1 to APIv2:

In [11]:
# Using APIv1

import requests
import json

url = "https://api.vectara.io/v1/query"
payload = json.dumps({
  "query": [
    {
      "query": query_str,
      "start": 0,
      "numResults": 10,
      "contextConfig": {
        "sentencesBefore": 2,
        "sentencesAfter": 2
      },
      "corpusKey": [
        {
          "customerId": customer_id,
          "corpusId": corpus_id,
          "lexicalInterpolationConfig": {
              "lambda": 0.025
          }
        }
      ],
      "summary": [
        {
          "maxSummarizedResults": 10,
          "responseLang": "eng",
          "summarizerPromptName": "vectara-summary-ext-24-05-sml",
          "factual_consistency_score": True
        }
      ]
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}
response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(res['responseSet'][0]['summary'][0]['text'])
print(f"Factual Consistency Score: {res['responseSet'][0]['summary'][0]['factualConsistency']['score']}")

Pets are not allowed at Vectara. While common household pets like cats and dogs are not permitted on the campuses, a select group of exotic creatures is welcomed instead to reflect the company's innovative spirit [1]. This unique approach includes interactions with exotic pets during team-building adventures and is a testament to Vectara's core values of innovation and unconventional thinking [6].
Factual Consistency Score: 0.654007


In [12]:
# Using APIv2

import requests
import json

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/query"

payload = json.dumps({
  "query": query_str,
  "search": {
    "lexical_interpolation": 0.025,
    "offset": 0,
    "limit": 10,
    "context_configuration": {
      "sentences_before": 2,
      "sentences_after": 2
    },
  },
  "generation": {
    "max_used_search_results": 10,
    "response_language": "eng",
    "prompt_name": "vectara-summary-ext-24-05-sml",
    "enable_factual_consistency_score": True
  }
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(res['summary'])
print(f"Factual Consistency Score: {res['factual_consistency_score']}")

Pets such as cats and dogs are not allowed at Vectara. However, the company welcomes a select group of exotic creatures as part of their pet policy, reflecting their innovative and daring spirit [1]. The exotic pets at Vectara serve as ambassadors of the company's vision and values, adding adventure and wisdom to the workplace [7].
Factual Consistency Score: 0.7290558


There a few important details to point out here:
1. First notice that our HTTP verb for APIv2 changed from a GET to a POST because we are creating something new. In this case, we are **creating a summary** of document information that answers our question, providing users a concise, compiled response that eliminates the need to read each retrieved text.

2. Note that in APIv2 the structure of the query JSON has been refactored to make it simple and more intuitive:
- The `query` string itself is a separate variable and we have `search` and `generation` as separate sub-dictionaries.
- Names have been changed: for example `maxSummarizedResults` -> `max_used_search_results`, `summarizerPromptName` -> `prompt_name` and `responseLang` -> `response_language`.
- `response_language` now only supports [ISO 693-3](https://iso639-3.sil.org/code_tables/639/data), so for example, for English you can only use `eng` (`en`, the 693-1 code won't work). When you migrate to APIv2 please use the playground and documentation to make sure you use the API in the way intended and notice any naming changes that are required.

3. The outputs in APIv2 are also easier to work with. For example the summary text is available as `res["summary"]` 

4. We also included the `factual_consistency_score` in our output, which provides the probability that the generated answer is factually consistent with the search results. You can learn about the factual consistency score [here](https://docs.vectara.com/docs/api-reference/search-apis/stream-query#factual-consistency-score).

5. Reranking is not used here in either APIv1 or APIv2. See below for further examples (e.g. in the Chat portion) that do enable the reranker.

## Streaming Output

One more thing about queries: You may want to be able to stream the summary of your query. To do this in APIv1, you have to use a completely separate API endpoint: [StreamQuery](https://docs.vectara.com/docs/1.0/rest-api/stream-query). We have greatly simplified this in APIv2 by creating a boolean flag `stream_response` to be used with both types of queries that allow generative summarization: [corpus query](https://docs.vectara.com/docs/rest-api/query-corpus) or [query corpora](https://docs.vectara.com/docs/rest-api/query) request. Let's see how this is done with a corpus query:

In [13]:
# Helper function to print streamed output for APIv1

def print_stream_v1(response):
    for line in response.iter_lines():
        line = line.decode("utf-8")
        if line:
            line = json.loads(line)
            if line['result']['summary']:
                print(line['result']['summary']['text'], end = '')
                if line['result']['summary']['factualConsistency']:
                    print(f"\nFactual Consistency Score: {line['result']['summary']['factualConsistency']['score']}")

In [14]:
# Using APIv1

import requests
import json

url = "https://api.vectara.io/v1/stream-query"

payload = json.dumps({
  "query": [
    {
      "query": query_str,
      "start": 0,
      "numResults": 10,
      "contextConfig": {
        "sentences_before": 2,
        "sentences_after": 2
      },
      "corpusKey": [
        {
          "corpus_id": corpus_id
        }
      ],
      "rerankingConfig": {
          "rerankerId": 272725718,
          "mmrConfig": {
            "diversityBias": 0.1
          }
      },
      "summary": [
        {
          "max_summarized_results": 10,
          "response_lang": "en",
          "summarizerPromptName": "vectara-summary-ext-24-05-sml",
          "factual_consistency_score": True
        }
      ]
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'customer-id': customer_id,
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload, stream=True)
print_stream_v1(response)

Pets are not allowed at Vectara. While common household pets like cats and dogs are not permitted on the campuses, Vectara welcomes a select group of exotic creatures that reflect their daring and innovative spirit [1]. This includes exotic animals like velociraptors, symbolizing agility, speed, and precision [9]. Additionally, Vectara has a virtual "Pet Corner" on their company intranet for sharing pictures, videos, and stories of cats and dogs [8].
Factual Consistency Score: 0.430898


In [15]:
# Helper function for printing out streamed response - API v2

def print_stream_v2(response):
    for line in response.iter_lines():
        line = line.decode("utf-8")
        if line:
            key, value = line.split(':', 1)
            if key == 'data':
                line = json.loads(value)
                if line['type'] == 'generation_chunk':
                    print(line['generation_chunk'], end = '')
                elif line['type'] == 'factual_consistency_score':
                    print(f"\nFactual Consistency Score: {line['factual_consistency_score']}")

In [16]:
# Using APIv2

import requests
import json

url = f"https://api.vectara.io/v2/corpora/{corpus_key}/query"

payload = json.dumps({
  "query": query_str,
  "search": {
    "lexical_interpolation": 0.025,
    "offset": 0,
    "limit": 10,
    "context_configuration": {
      "sentences_before": 2,
      "sentences_after": 2
    },
    "reranker": {
      "type": "mmr",
      "diversity_bias": 0.1
    }
  },
  "generation": {
    "max_used_search_results": 10,
    "response_language": "eng",
    "prompt_name": "vectara-summary-ext-24-05-sml",
    "enable_factual_consistency_score": True
  },
  "stream_response": True
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'text/event-stream',
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload, stream=True)
print_stream_v2(response)

Pets are not allowed at Vectara. While common household pets like cats and dogs are not permitted on the Vectara campuses, a select group of exotic creatures is welcomed instead to reflect the company's innovative spirit [1]. This unique approach includes hosting exotic animals in an on-site mini-zoo for inspiration, relaxation, and education [4]. The company values innovation, courage, and unconventional thinking, which is reflected in their choice of exotic pets [7].
Factual Consistency Score: 0.6742027


Notice that we have added something else to our query: a **reranker**. After the initial search query, it is often useful to re-sort to retrieved results to achieve a more precise reranking. 

There are two types of rerankers:
1. [Maximal Marginal Relevance](https://docs.vectara.com/docs/api-reference/search-apis/reranking#maximal-marginal-relevance-mmr-reranker), or MMR, reranker, which can be used to increase diversity of results instead of focusing solely on relevance. The `diversity_bias` parameter, which ranges from 0 to 1, specifies the extent to which diversity is requested (0=full relevance, 1=full diversity).
2. [Multilingual Reranker](https://docs.vectara.com/docs/api-reference/search-apis/reranking#vectara-multilingual-reranker-v1), which can be used to rerank by relevance to the query, and supports reranking across 100+ languages.

Notice that in APIv2, the `reranker` parameter is contained within the `search` part of our query and is completed before generating the summarized results.

## Vectara Chat

The Chat functionality allows you to have conversations with your data and ask multiple related queries without re-specifying all of the details from your previous queries. This means Vectara maintains history of your chat question/response pairs in the conversation and uses that for future questions to provide more context.

When using chat, your first query will create a new chat and the response will include a generated `conversationId`. After this, you will use the `conversationId` for all subsequent queries, known as turns. Let's take a look at the difference between chat conversations in APIv1 and APIv2:

In [17]:
def chat_api_v1(query, conversation_id=""):
    url = "https://api.vectara.io/v1/query"
    
    payload = json.dumps({
      "query": [
        {
          "query": query,
          "start": 0,
          "numResults": 10,
          "contextConfig": {
            "sentencesBefore": 2,
            "sentencesAfter": 2
          },
          "corpusKey": [
            {
              "customerId": customer_id,
              "corpusId": corpus_id,
              "lexicalInterpolationConfig": {
                "lambda": 0.025
              }
            }
          ],
          "rerankingConfig": {
              "rerankerId": 272725718,
              "mmrConfig": {
                "diversityBias": 0.1
              }
          },
          "summary": [
            {
              "maxSummarizedResults": 5,
              "responseLang": "en",
              "summarizerPromptName": "vectara-summary-ext-24-05-sml",
              "chat": {
                "store": True,
                "conversationId": conversation_id,
                "factualConsistencyScore": True
              }
            }
          ]
        }
      ]
    })
    
    headers = {
      'Content-Type': 'application/json',
      'Accept': 'application/json',
      'customer-id': customer_id,
      'x-api-key': corpus_api_key
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    res = response.json()
    print(res['responseSet'][0]['summary'][0]['text'])
    print(f"Factual Consistency Score: {res['responseSet'][0]['summary'][0]['factualConsistency']['score']}\n")
    return res['responseSet'][0]['summary'][0]['chat']['conversationId']

In [18]:
# Using APIv1

import requests
import json

# Creating a chat conversation
conv_id_v1 = chat_api_v1(query="Are velociraptors allowed in the office?")

# Chat turn
_ = chat_api_v1(query="What makes them special?", conversation_id=conv_id_v1)

Velociraptors are allowed in the office as they symbolize agility, speed, and precision, serving as a reminder to approach work with ferocity and focus. Trained professionals handle them, and they even act as excellent security guards [1]. Common household pets like cats and dogs are not permitted, but a select group of exotic creatures, including velociraptors, reflecting a daring and innovative spirit, are welcomed [2].
Factual Consistency Score: 0.9072879

Velociraptors are special due to their symbolization of agility, speed, and precision. They serve as a constant reminder to approach work with ferocity and focus, making them excellent security guards. Trained professionals handle them, highlighting their unique characteristics [1].
Factual Consistency Score: 0.908545



In [19]:
def chat_api_v2(query, conversation_id=""):
    if conversation_id == "":
        url = "https://api.vectara.io/v2/chats"
    else:
        url = f"https://api.vectara.io/v2/chats/{conversation_id}/turns"

    payload = json.dumps({
      "query": query,
      "search": {
        "corpora": [
          {
            "lexical_interpolation": 0.025,
            "corpus_key": corpus_key
          }
        ],
        "offset": 0,
        "limit": 10,
        "context_configuration": {
          "sentences_before": 2,
          "sentences_after": 2
        },
        "reranker": {
          "type": "mmr",
          "diversity_bias": 0.1
        }
      },
      "generation": {
        "prompt_name": "vectara-summary-ext-24-05-sml",
        "max_used_search_results": 5,
        "response_language": "eng",
        "enable_factual_consistency_score": True
      },
      "chat": {
        "store": True
      },
      "stream_response": False
    })
    headers = {
      'Content-Type': 'application/json',
      'Accept': 'application/json',
      'x-api-key': corpus_api_key
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    res = response.json()
    print(res['answer'])
    print(f"Factual Consistency Score: {res['factual_consistency_score']}\n")
    return res['chat_id']

In [20]:
# Using APIv2

import requests
import json

# Creating a chat conversation
conv_id_v2 = chat_api_v2(query="Are velociraptors allowed in the office?")


# Chat turn
_ = chat_api_v2(query="What makes them special?", conversation_id=conv_id_v2)

Velociraptors are allowed in the office as they symbolize agility, speed, and precision, serving as a reminder to approach work with ferocity and focus. Trained professionals handle them, and they even make excellent security guards [1]. Common household pets like cats and dogs are not permitted, but a select group of exotic creatures, including velociraptors, reflecting a daring and innovative spirit, are welcomed [2].
Factual Consistency Score: 0.9060313

Velociraptors are special due to their symbolization of agility, speed, and precision. They serve as a constant reminder to approach work with ferocity and focus, making them excellent security guards. Trained professionals handle them, highlighting their unique characteristics [1].
Factual Consistency Score: 0.908545



Some key takeaways:
1. First notice that in the second turn, we asked "What makes **them** special?". Based on our first turn, "Are *velociraptors* allowed in the office?", Vectara knows that **them** refers to *velociraptors*. However, if you look at the two responses, APIv1's response talks about multiple animals from the initial response, such as alligators while APIv2's response is correctly focused on velociraptors, providing a more detailed explanation.
2. Take a look at the urls for each request. In APIv1, we are still using a query, just as we did for the simple corpus query and corpus query. We differentiate a chat from other query types by adding a `chat` parameter in the `summmary` section. Furthermore, we differentiate the creation of a new chat and a chat turn by the parameter `conversationId`. In contrast, APIv2 distinguishes chats in the request url, making it clearer that this is a turn in a chat and not an individual query.

## Authentication: Creating an Application Client

Vectara supports authentication using both OAUTH2 and API keys. 

Although considered more complex, in many cases OAUTH2 is preferred as it provides additional layers of security (e.g. the tokens expire after 30 minutes).

To use OAUTH2 with Vectara, you need to follow these steps:
1. Create an "Application Client"
2. Generate a JWT Token
3. Use the JWT Token in a Vectara API request

Let's start with creating an application client in both APIv1 and APIv2. There are many different roles of app clients that you can create, `owner`, `administrator`, `billing_administrator`, and `corpus_administrator`. You can read more about each of the app clients and their use cases [here](https://docs.vectara.com/docs/1.0/learn/authentication/role-based-access-control).

Using APIv1: You may notice in our API Playground that there is no page for creating an app client, and yes it's not supported in APIv1. That being said, you can still create an API client through your Console under the authorization tab. Navigate to the [OAuth 2.0 tab](https://console.vectara.com/console/apiAccess/appClients) and you will see a blue button that says "Create App Client". You can create your app client through this interface.

Creating API clients in the console works regardless of which version of the API you use, but with APIv2, you can also create an app client programmatically with the code below:

In [21]:
# Using APIv2

import requests
import json

url = "https://api.vectara.io/v2/app_clients"

payload = json.dumps({
  "name": "corpus_admin",
  "description": "This app client can do all corpus-related actions",
  "type": "client_credentials",
  "api_roles": [
    "corpus_administrator"
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'x-api-key': personal_api_key
}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(f"App Client ID: {res['id']}\nApp Client Name: {res['name']}\nApp Client Description: {res['description']}\n\nApp Client Permissions:")
for key, value in res['api_policy']['allowed_operations'].items():
    print(f"Operation Category: {key}; Client Permissions: {value}")

App Client ID: app_5
App Client Name: corpus_admin
App Client Description: 

App Client Permissions:
Operation Category: getChat; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: getJob; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: replaceFilterAttributes; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: uploadFile; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: createChat; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: listRerankers; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: updateChatTurn; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: listLLMs; Client Permissions: {'allow_any_resource': True, 'allowed_resources': {}}
Operation Category: queryCorpus; Client 

## Generating a JWT Token

The next step in this process is to generate a JWT Token. This is done in the same manner for both APIv1 and APIv2 by running the following cURL command:

```Python
curl -XPOST -H "Content-type: application/x-www-form-urlencoded" 
     -d "grant_type=client_credentials&client_id=\<your client ID goes here>&client_secret=\<your client secret goes here>” “\<your authentication URL goes here>”
```

Don't forget to replace all of the angle bracket info with the correct info from your app client. You can get all of this info in APIv2 with a GET request:

In [22]:
app_client_id = res['id']

In [23]:
import requests

url = f"https://api.vectara.io/v2/app_clients/{app_client_id}"

payload={}
headers = {
  'Accept': 'application/json',
  'x-api-key': personal_api_key
}

response = requests.request("GET", url, headers=headers, data=payload)
res = response.json()

# Uncomment this code to see information for your client
# print(f"Client ID: {res['id']}")
# print(f"Client Secret: {res['client_secret']}")
# print("Authentication URL: https://auth.vectara.io/oauth2/token")

This cURL should generate a JWT token that you can now use instead of a API key to authorize your other requests.

## Displaying Errors

Another important change from APIv1 to APIv2 is the way errors are communicated back from the API.

For example, let's see what happens when we try to query a corpus that does not exist:

In [24]:
# Using APIv1

import requests
import json

url = "https://api.vectara.io/v1/query"

payload = json.dumps({
  "query": [
    {
      "query": "How can I write a query?",
      "start": 0,
      "numResults": 10,
      "corpusKey": [
        {
          "customerId": customer_id,
          "corpusId": 12
        }
      ]
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'customer-id': customer_id,
  'x-api-key': corpus_api_key
}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.json()
print(response.status_code)
print(res['responseSet'][0]['status'][0])

200
{'code': 'QRY__INVALID_CORPUS', 'statusDetail': 'Cannot resolve corpus: 12', 'cause': None}


In [25]:
# Using APIv2

import requests

url = "https://api.vectara.io/v2/corpora/vectara_docs/query?query=How%20can%20I%20write%20a%20query?"

payload={}
headers = {
  'Accept': 'application/json',
  'x-api-key': corpus_api_key
}

response = requests.request("GET", url, headers=headers, data=payload)
res = response.json()
print(response.status_code)
print(res['messages'][0])

403
Denied access to resource vectara_docs.


Some takeaways:
1. The `status_code` in the response indicates whether or not the request was successful, and if not, where the issue occurred:
   - A response code in the 200's indicates that the request was successful
   - A response code in the 400's indicates that the request was unsuccessful due to an issue with the request body (often a syntax error in the request)
   - A response code in the 500's indicates that the request was unsuccessful due to an issue at the server level
   
2. APIv1 breaks up the response into a `code`, `statusDetail`, and `cause`, and in some cases response with a 200 code only to have the actual status code and text inside the `statusDetail` section. The response from APIv2 is more succinct than APIv1, providing a single, easy to understand message, and adhering correctly to status_code best practices.